In [9]:
pip install openai==0.28

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.34.0
    Uninstalling openai-1.34.0:
      Successfully uninstalled openai-1.34.0

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import openai
import pandas as pd
import os
import re

# Set your OpenAI API key here
openai.api_key = 'xxx
# Function to evaluate the likelihood that an answer is by Rick
def evaluate_answer(question, answer):
    prompt = f"Question: {question}\nAnswer: {answer}\n\nBased on the language and style of the answer, give a score from 1 to 100 indicating how likely it is that the answer is given by Rick from 'Rick and Morty'. Provide the score."
    
    # Use the latest API method for creating completions
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100
    )
    result = response['choices'][0]['message']['content'].strip()
    return result

# Change directory to where your CSV file is located
os.chdir(os.path.expanduser('~/Downloads'))

# Read the CSV file
file_path = 'filled_answers__unfinetuned_distilgpt2.csv'  # Replace with your CSV file path
df = pd.read_csv(file_path)

# Ensure the CSV has the correct columns
if 'Question' not in df.columns or 'Answer' not in df.columns:
    raise ValueError("CSV file must contain 'Question' and 'Answer' columns.")

# Evaluate each answer and store the results
scores = []
for index, row in df.iterrows():
    question = row['Question']
    answer = row['Answer']
    evaluation = evaluate_answer(question, answer)
    scores.append(evaluation)

# Add the scores to the DataFrame
df['Rick_Score'] = scores

# Save the results to a new CSV file
output_file_path = 'questions_answers_with_scores.csv'
df.to_csv(output_file_path, index=False)

print(f"Scores saved to {output_file_path}")

In [24]:
import openai
import pandas as pd
import os
import time
import re
# Set your OpenAI API key here
openai.api_key = 'sk-proj-FF3S538AbSJLTCvm1EoXT3BlbkFJ2wCnazhpen7wd5xZGGXt'

# Function to evaluate the likelihood that an answer is by Rick
def evaluate_answer(question, answer):
    prompt = f"Question: {question}\nAnswer: {answer}\n\nBased on the language and style of the answer, give a score from 1 to 100 indicating how likely it is that the answer is given by Rick from 'Rick and Morty'. Provide the score."
    
    while True:
        try:
            # Use the latest API method for creating completions
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=100
            )
            result = response['choices'][0]['message']['content'].strip()
            return result
        except openai.error.RateLimitError:
            print("Rate limit exceeded. Waiting for 60 seconds before retrying...")
            time.sleep(60)  # Wait for 60 seconds before retrying
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

# Change directory to where your CSV file is located
os.chdir(os.path.expanduser('~/Downloads/Inference'))

# Read the combined CSV file
file_path = 'combined_answers.csv'  # Replace with your CSV file path
df = pd.read_csv(file_path)

# Ensure the CSV has the correct columns
if 'Question' not in df.columns or not any(col.startswith('Answer_') for col in df.columns):
    raise ValueError("CSV file must contain 'Question' and 'Answer_' columns.")

# Add columns for scores
for col in df.columns:
    if col.startswith('Answer_'):
        df[f'{col}_Score'] = None
        
# Evaluate answers in batches of 10 to respect rate limits
batch_size = 10
for start in range(0, len(df), batch_size):
    end = min(start + batch_size, len(df))
    batch = df.iloc[start:end]

    # Loop through each row in the batch and evaluate each answer
    for index, row in batch.iterrows():
        question = row['Question']
        for col in df.columns:
            if col.startswith('Answer_'):
                answer = row[col]
                if pd.notna(answer):  # Check if the answer is not NaN
                    evaluation = evaluate_answer(question, answer)
                    score_column = f"{col}_Score"
                    df.at[index, score_column] = evaluation

# Save the results to a new CSV file
output_file_path = 'questions_answers_with_scores.csv'
df.to_csv(output_file_path, index=False, encoding='utf-8')

print(f"Scores saved to {output_file_path}")

Scores saved to questions_answers_with_scores.csv
